# Adult Data Set

In this problem, we are going to fit a model on the adult data set to predict which people have income over $50K.

Link to the data set: https://archive.ics.uci.edu/ml/datasets/adult

In [66]:
using Random
Random.seed!(13)

using Plots, DataFrames, CSV

using LinearAlgebra, Statistics
using DecisionTree
using ScikitLearn
@sk_import linear_model: LogisticRegression
@sk_import metrics: accuracy_score

PyObject <function accuracy_score at 0x1a4cd99488>

In [67]:
# The data set contains a large number of features relevant to income prediction.
# First, let's label our features and target value 'Income'.

labels = [:Age, :Workclass, :fnlwgt, :Education, :Education_num, :Marital_Status, :Occupation, :Relationship, :Race, :Sex, :Capital_Gain, :Capital_Loss, :Hours_Per_Week, :Native_Country, :Income]

15-element Array{Symbol,1}:
 :Age           
 :Workclass     
 :fnlwgt        
 :Education     
 :Education_num 
 :Marital_Status
 :Occupation    
 :Relationship  
 :Race          
 :Sex           
 :Capital_Gain  
 :Capital_Loss  
 :Hours_Per_Week
 :Native_Country
 :Income        

In [68]:
# Next, let's load the data.
data = CSV.read("adult_income.csv"; header=labels, copycols=true)

for i in 1:length(labels)
    println(string(i), "\t", string(labels[i]), "\t\t\t", string(eltype(data[!, i])))
end

1	Age			String
2	Workclass			String
3	fnlwgt			Int64
4	Education			String
5	Education_num			Int64
6	Marital_Status			String
7	Occupation			String
8	Relationship			String
9	Race			String
10	Sex			String
11	Capital_Gain			Int64
12	Capital_Loss			Int64
13	Hours_Per_Week			Int64
14	Native_Country			String
15	Income			String


## Target Value
Create a feature attribute for Income as a Boolean. 

In [69]:
# Notice that income is given as a string; we'll change it to a boolean.
data[:, :Income]

32561-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " >50K" 
 " >50K" 
 " >50K" 
 " >50K" 
 " >50K" 
 " <=50K"
 ⋮       
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " <=50K"
 " >50K" 
 " <=50K"
 " <=50K"
 " >50K" 
 " <=50K"
 " <=50K"
 " >50K" 

In [70]:
# Here, we change income into an integer value. 
# The value '1' indicates income over $50K. 
# The value '0' indicates income less than or equal to $50K.

data.Income_Bool = data.Income .== " >50K"
data.Income_Integer = convert(Array{Float64}, data[:, :Income_Bool])

32561-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0
 0.0
 0.0
 1.0

In [71]:
# Here's our new set of features (and labels).
push!(labels, :Income_Bool)  
push!(labels, :Income_Integer)  

17-element Array{Symbol,1}:
 :Age           
 :Workclass     
 :fnlwgt        
 :Education     
 :Education_num 
 :Marital_Status
 :Occupation    
 :Relationship  
 :Race          
 :Sex           
 :Capital_Gain  
 :Capital_Loss  
 :Hours_Per_Week
 :Native_Country
 :Income        
 :Income_Bool   
 :Income_Integer

## Train/Test Split

Let's create a training and test set by splitting the data at random.

In [72]:
data = data[.!(ismissing.(data[!, :Income])), :]
data = data[shuffle(1:end), :]

train_proportion = 0.8
n = size(data, 1)

println("Size of dataset: ", string(n))

ntrain = convert(Int, round(train_proportion*n))

target = data[:, :Income_Integer]
income_data = data[:, filter(col -> (col != :Income && col!= :Income_Bool && col!= :Income_Integer), labels)]

train_x = income_data[1:ntrain, :]
test_x = income_data[ntrain+1:end, :]
train_y = target[1:ntrain, :]
test_y = target[ntrain+1:end, :]

train_x

Size of dataset: 32561


,Age,Workclass,fnlwgt,Education,Education_num,Marital_Status
,String,String,Int64,String,Int64,String
1,41,Local-gov,523910,Bachelors,13,Married-civ-spouse
2,53,State-gov,229465,Doctorate,16,Married-civ-spouse
3,48,Private,233511,HS-grad,9,Married-civ-spouse
4,51,Self-emp-not-inc,118259,HS-grad,9,Married-civ-spouse
5,29,Private,200511,Bachelors,13,Married-civ-spouse
6,40,Private,116103,Some-college,10,Separated
7,42,Private,175526,HS-grad,9,Married-civ-spouse
8,39,Private,141802,Some-college,10,Divorced
9,40,Self-emp-not-inc,29036,Some-college,10,Never-married


## Real-Valued Data

Let's identify the real-valued parameters.

  - Education_num
  - Capital_Gain
  - Capital_Status
  - Hours_Per_Week
  
Also it's important to include the following parameter stored as a string.

  - age
  
Use a utility function to interpret any non-numerical value as "0", which is an accurate interpretation of "N/A" in this context.

In [73]:
# What does education num represent?

d = Dict(zip(train_x[:Education_num], train_x[:Education]))
ednum = sort(collect(keys(d)))
for i in ednum
    print("$i: $(d[i])\n")
end

1:  Preschool
2:  1st-4th
3:  5th-6th
4:  7th-8th
5:  9th
6:  10th
7:  11th
8:  12th
9:  HS-grad
10:  Some-college
11:  Assoc-voc
12:  Assoc-acdm
13:  Bachelors
14:  Masters
15:  Prof-school
16:  Doctorate


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[73]:1
└ @ Core In[73]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[73]:1
└ @ Core In[73]:1


In [74]:
"This function converts strings to floating point values and strings that cannot be 
represented as a number (such as N/A) are converted to zeros"

function string_to_float(str)
    try
        parse(Float64, str)
    catch
        0.0
    end
end

labels_real = [
    :Education_num,
    :Capital_Gain,
    :Capital_Loss,
    :Hours_Per_Week
]


labels_str = [
    :Age
]

1-element Array{Symbol,1}:
 :Age

In [75]:
# Here, we extract the specific columns related to these labels, convert them to floats as necessary, and convert the result into arrays.
train_vals_real = convert(Matrix, train_x[:, labels_real])
test_vals_real = convert(Matrix, test_x[:, labels_real])

train_vals_from_str = hcat(string_to_float.(train_x[:, :Age]))
test_vals_from_str = hcat(string_to_float.(test_x[:, :Age]))

@assert(eltype(train_vals_from_str) != String)
@assert(eltype(test_vals_from_str) != String)

# Concatenate real-valued data into a matrix. The train/test dataset do not include the protected attribute 'sex'.
# We'll need them to evaluate the unawareness fairness metric.

train_ua = hcat(train_vals_real, train_vals_from_str)
test_ua = hcat(test_vals_real, test_vals_from_str)

6512×5 Array{Float64,2}:
 13.0     0.0    0.0  40.0  35.0
 14.0     0.0    0.0  40.0  36.0
 13.0     0.0  625.0  40.0  43.0
  9.0     0.0    0.0  30.0  48.0
  7.0     0.0    0.0  40.0  34.0
  2.0     0.0    0.0  15.0  81.0
 13.0     0.0    0.0  70.0  35.0
  9.0  4101.0    0.0  40.0  31.0
  6.0     0.0    0.0  40.0  33.0
  7.0     0.0    0.0  40.0  59.0
 13.0     0.0    0.0  16.0  23.0
 10.0     0.0    0.0  30.0  20.0
 13.0     0.0    0.0  40.0  30.0
  ⋮                             
 16.0     0.0    0.0  55.0  33.0
  7.0     0.0    0.0  30.0  18.0
 10.0     0.0    0.0  16.0  18.0
 10.0     0.0    0.0  40.0  24.0
 14.0     0.0    0.0  40.0  57.0
  4.0     0.0    0.0  45.0  19.0
  9.0     0.0    0.0  40.0  24.0
  8.0     0.0    0.0  30.0  18.0
 10.0     0.0    0.0  40.0  32.0
 16.0     0.0    0.0  40.0  27.0
  5.0     0.0    0.0  40.0  35.0
 13.0   991.0    0.0  10.0  90.0

In [76]:
# Here is a list of the non-protected features that we'll use to train our models.
train_features = vcat(labels_real, labels_str)

5-element Array{Symbol,1}:
 :Education_num 
 :Capital_Gain  
 :Capital_Loss  
 :Hours_Per_Week
 :Age           

## Boolean data – Sex (Protectected Attribute)
Let's stratify our data based on the protected attribute.

In [77]:
data[:, :Sex]

32561-element PooledArrays.PooledArray{String,UInt32,1,Array{UInt32,1}}:
 " Male"  
 " Male"  
 " Male"  
 " Male"  
 " Male"  
 " Male"  
 " Male"  
 " Female"
 " Male"  
 " Female"
 " Male"  
 " Male"  
 " Female"
 ⋮        
 " Male"  
 " Female"
 " Male"  
 " Male"  
 " Female"
 " Male"  
 " Male"  
 " Female"
 " Male"  
 " Female"
 " Male"  
 " Female"

In [78]:
#Let's convert the strings " Male" and " Female" to Boolean values and then convert these Boolean values into integers.
# The value '1' indicates that a person identifies as a 'Male'. 
# The value '0' indicates that a person identifies as a 'Female' or 'Other'. 

train_vals_sex = convert(Array{Float64}, hcat(train_x[:, :Sex] .== " Male"))
test_vals_sex =  convert(Array{Float64}, hcat(test_x[:, :Sex] .== " Male"))

6512×1 Array{Float64,2}:
 1.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 ⋮  
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0
 0.0

## Train and Test datasets for Male and Female examples
We need a train and test dataset for both the male and female examples so that we can evaluate the fairness of our models.

In [79]:
# We need to add the protected feature, 'sex', to our train and test datasets.

# Here is a list of features (including the protected attribute) that we'll use to train our models.
train_features_w_sex = vcat(labels_real, labels_str, :Sex)

train_vals = hcat(train_vals_real, train_vals_from_str, train_vals_sex)
test_vals = hcat(test_vals_real, test_vals_from_str, test_vals_sex)

# We need the index of the 'sex' feature. It's the last column in our train and test datasets.
sex_id = size(train_vals, 2)

# These are the male examples in our train and test samples.
train_male = train_vals[train_vals[:, sex_id] .== 1.0, :]
test_male = test_vals[test_vals[:, sex_id] .== 1.0, :]

# These are the female examples in our train and test samples.
train_female = train_vals[train_vals[:, sex_id] .== 0.0, :]
test_female = test_vals[test_vals[:, sex_id] .== 0.0, :]

2085×6 Array{Float64,2}:
 14.0    0.0    0.0  40.0  36.0  0.0
 13.0    0.0  625.0  40.0  43.0  0.0
  9.0    0.0    0.0  30.0  48.0  0.0
 13.0    0.0    0.0  16.0  23.0  0.0
 10.0    0.0    0.0  30.0  20.0  0.0
 13.0    0.0    0.0  45.0  33.0  0.0
  9.0    0.0    0.0  40.0  19.0  0.0
  9.0    0.0    0.0  55.0  37.0  0.0
  9.0    0.0    0.0  35.0  18.0  0.0
 12.0    0.0    0.0  38.0  43.0  0.0
  9.0    0.0    0.0  20.0  19.0  0.0
 14.0    0.0    0.0  50.0  47.0  0.0
  9.0    0.0    0.0  40.0  18.0  0.0
  ⋮                              ⋮  
 10.0    0.0    0.0  40.0  37.0  0.0
 13.0    0.0    0.0  36.0  37.0  0.0
 10.0    0.0    0.0  40.0  39.0  0.0
  6.0    0.0    0.0  25.0  29.0  0.0
 10.0    0.0    0.0  40.0  29.0  0.0
  7.0    0.0    0.0  40.0  44.0  0.0
  9.0    0.0    0.0  32.0  49.0  0.0
  7.0    0.0    0.0  30.0  18.0  0.0
 14.0    0.0    0.0  40.0  57.0  0.0
  8.0    0.0    0.0  30.0  18.0  0.0
 16.0    0.0    0.0  40.0  27.0  0.0
 13.0  991.0    0.0  10.0  90.0  0.0

Notice that every value in the last column of test_female is '0.0'.

## Target values for the Male and Female train and test datasets

In [80]:
# We need to match up the target values for our male and female train and test examples.

# Let's concatenate the 'sex' feature with the target label. We have to do this for both the train and test datasets.
t1 = hcat(train_vals_sex, train_y)
t2 = hcat(test_vals_sex, test_y)

# We need the index of the 'sex' feature in our t1 and t2 sets. It's the first column.
t_sex_index = 1

# We need the index of the target value in our t1 and t2 sets. It's the last column.
t_target_index = size(t1, 2)


# These are the target values for male and female training samples.
train_male_y = t1[t1[:, t_sex_index] .== 1.0, t_target_index]
train_female_y = t1[t1[:, t_sex_index] .== 0.0, t_target_index]


# These are the target values for male and female test samples.
test_male_y = t2[t2[:, t_sex_index] .== 1.0, t_target_index]
test_female_y = t2[t2[:, t_sex_index] .== 0.0, t_target_index]


2085-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# Now, we are ready to fit a model to our data.
We'll use Logistic regression as our baseline classifier. # These are the target values for male and female training samples.

### Let's see how our classifier performs using only the sex feature (protected attribute).

In [81]:
logistic_sex_only = fit!(LogisticRegression(), train_vals_sex, train_y).predict(test_vals_sex)

# This tells us how well we can predict income based on 'sex' only.
accuracy_score(test_y, logistic_sex_only)

0.7587530712530712

This model is over 75% accurate. Sex predicts income *too* well!

## Fairness Metric: Unawareness
We need to compare the accuracy of the model when including and NOT including the protected attribute.


In [82]:
# This model does not take into account the 'sex' feature.
logistic_without_sex = fit!(LogisticRegression(), train_ua, train_y).predict(test_ua)
accuracy_score(test_y, logistic_without_sex)

0.8132678132678133

In [83]:
# This model does include the protected attribute, 'sex'.
logistic_with_sex = fit!(LogisticRegression(), train_vals, train_y).predict(test_vals)
accuracy_score(test_y, logistic_with_sex)

0.8183353808353808

The accuracy for both datasets is around 81-82%. This means that 'sex' does not have a substantial effect on our predictions.

## Before we move on to other fairness metrics, let's plot some results.
Recall that 'train_features_w_sex' is a list of features (including the protected attribute) that we used to train our model.

In [ ]:
ypred = 2*logistic_with_sex.-1

# We choose two features to plot.
features_to_plot = [1,4]

xlabel = String(train_features_w_sex[features_to_plot[1]])
ylabel = String(train_features_w_sex[features_to_plot[2]])

plot()

# plot negative examples 
neg_ex = ypred .== -1
scatter!(test_vals[neg_ex, features_to_plot[1]], test_vals[neg_ex, features_to_plot[2]], color = :red, label="negative", alpha=.3)

# plot positive examples
pos_ex = ypred .== 1
scatter!(test_vals[pos_ex, features_to_plot[1]], test_vals[pos_ex, features_to_plot[2]], color = :blue, label="positive", alpha=.3)

xlabel!(xlabel)
ylabel!(ylabel)

## Fairness Metric: Statistical Parity

In [ ]:
# Let's do Logistic regression on the male examples.
logistic_male = fit!(LogisticRegression(), train_male, train_male_y).predict(test_male)

# Let's calculate the percentage of males predicted to have an income > $50K. 
logistic_sp_male = sum(logistic_male) / length(logistic_male)

In [ ]:
# Let's do Logistic regression on the female examples.
logistic_female = fit!(LogisticRegression(), train_female, train_female_y).predict(test_female)

# Let's calculate the percentage of females predicted to have an income > $50K.  
logistic_sp_female = sum(logistic_female) / length(logistic_female)

Males are about 7x more likely to have an income > $ 50K than females. That's a huge discrepancy!

## Fairness Metric: Equalized Odds

Here are some helper functions which help us compute the True Positive rate, True Negative rate, False Positive rate, and False Negative rate. We'll need these values to calculate equalized odds for this dataset.

In [ ]:
# Count the number of true positives.
function TP(actual, prediction)
    counter = 0
    for i in 1:length(actual)
        if (actual[i] ==1 && prediction[i] ==1)
            counter += 1
        end
    end
    return counter
end

# This function calculates the True Positive rate.
function TPR(actual, prediction)
    TP(actual, prediction) / sum(actual)
end

In [ ]:
# Count the number of true negatives.
function TN(actual, prediction)
    counter = 0
    for i in 1:length(actual)
        if (actual[i] ==0 && prediction[i] ==0)
            counter += 1
        end
    end
    return counter
end

# This function calculates the True Negative rate.
function TNR(actual, prediction)
    TN(actual, prediction) / (length(actual) - sum(actual))
end

In [ ]:
# Count the number of false positives.
function FP(actual, prediction)
    counter = 0
    for i in 1:length(actual)
        if (actual[i] ==0 && prediction[i] ==1)
            counter += 1
        end
    end
    return counter
end

# This function calculates the False Positive rate.
function FPR(actual, prediction)
    FP(actual, prediction) / (length(actual) - sum(actual))
end

In [ ]:
# Count the number of false negatives.
function FN(actual, prediction)
    counter = 0
    for i in 1:length(actual)
        if (actual[i] ==1 && prediction[i] ==0)
            counter += 1
        end
    end
    return counter
end

# This function calculates the False Negative rate.
function FNR(actual, prediction)
    FN(actual, prediction) / sum(actual)
end

#### True Positives (Equality of Opportunity)

In [ ]:
TPR_log_male = TPR(test_male_y,logistic_male)

In [ ]:
TPR_log_female = TPR(test_female_y, logistic_female)

Men are over 2x more likely to be correctly classified with income over $50K than women.

#### True Negatives

In [ ]:
TNR_log_male = TNR(test_male_y, logistic_male)

In [ ]:
TNR_log_female = TNR(test_female_y, logistic_female)

#### False Positive

In [ ]:
FPR_log_male = FPR(test_male_y, logistic_male)

In [ ]:
FPR_log_female = FPR(test_female_y, logistic_female)

#### False Negatives

In [ ]:
FNR_log_male = FNR(test_male_y, logistic_male)

In [ ]:
FNR_log_female = FNR(test_female_y, logistic_female)

###  Predictive Rate Parity
We can use Bayes Theorem to figure out the Predictive Rate Parity for males and females.

In [ ]:
# Positive Predictive Rate Parity for males
PPR_male = (TPR_log_male * sum(test_male_y)) / sum(logistic_male)

In [ ]:
# Positive Predictive Rate Parity for females
PPR_female = (TPR_log_female * sum(test_female_y)) / sum(logistic_female)

In [ ]:
# Negative Predictive Rate Parity  for males
NPR_male = (TNR_log_male * (length(test_male_y) - sum(test_male_y))) / (length(logistic_male) - sum(logistic_male))

In [ ]:
# Negative Predictive Rate Parity  for females
NPR_female = (TNR_log_female * (length(test_female_y) - sum(test_female_y))) / (length(logistic_female) - sum(logistic_female))

# Let's compare our predictions from Logistic regression with Random Forest.

We'll look at Unawareness, Statistical Parity, Equalized odds, and Predictive Rate Parity for our Random Forest classifier.


## Fairness Metric: Unawareness (using Random Forest)
We need to compare the accuracy of the model when including and NOT including the protected attribute.

In [ ]:
# Here we fit a random forest classifier on our dataset with includes the protected attribute.
rf_clf_with_sex = fit!(RandomForestClassifier(n_trees=20, n_subfeatures = 4), train_vals, vec(train_y))

# Next, we make predictions with our random forest classifier.
rf_with_sex = predict(rf_clf_with_sex, test_vals)

As you can see, our predictions take on values between '0' and '1'. We'll set a threshold for our predictions.

In [ ]:
rf_bin_with_sex = rf_with_sex .> .65

Now we get '0' or '1'.

In [ ]:
accuracy_score(test_y, rf_bin_with_sex)

In [ ]:
# Here, we fit a random forest classifier on our dataset. This dataset does not include the protected attribute, 'sex'.
rf_clf_without_sex = fit!(RandomForestClassifier(n_trees=20, n_subfeatures = 4), train_ua, vec(train_y))

# Next, we make predictions with our random forest classifier.
rf_without_sex = predict(rf_clf_without_sex, test_ua)

# Set a threshold for our predictions.
rf_bin_without_sex = rf_without_sex .> .65

accuracy_score(test_y, rf_bin_without_sex)

We approximately get the same accuracy score for both, so the protected attribute, sex, does not influence our predictions.

### Let's see how our classifier performs using only the sex feature (protected attribute).

In [ ]:
# Here, we change the number of subfeatures to be 1 since we are only using one feature.
rf_clf_sex_only = fit!(RandomForestClassifier(n_trees=20, n_subfeatures = 1), train_vals_sex, vec(train_y))
rf_sex_only = predict(rf_clf_sex_only, test_vals_sex)

rf_bin_sex_only = rf_sex_only .> .65

# This tells us how well we can predict income based on sex only.
accuracy_score(test_y, rf_bin_sex_only)

This model is over 75% accurate. Sex predicts income *too* well! We saw the same value for Logistic regression.

## Fairness Metric: Statistical Parity (using Random Forest)

In [ ]:
# Random Forest on male examples
rf_clf_male = fit!(RandomForestClassifier(n_trees=20, n_subfeatures = 4), train_male, vec(train_male_y))
rf_male = predict(rf_clf_male, test_male)

rf_male_bin = rf_male .> .65

# Let's calculate the percentage of males predicted to have an income > $50K.
rf_sp_male = sum(rf_male_bin) / length(rf_male_bin)

In [ ]:
# Random Forest on female examples
rf_clf_female = fit!(RandomForestClassifier(n_trees=20, n_subfeatures = 4), train_female, vec(train_female_y))
rf_female = predict(rf_clf_female, test_female)
rf_female_bin = rf_female .> .65


# Let's calculate the percentage of females predicted to have an income > $50K. 
rf_sp_female = sum(rf_female_bin) / length(rf_female_bin)

Males are about 4.5x more likely to have a predicted income > $ 50K than females.

In [ ]:
# will delete
rf_sp_diff = rf_sp_male - rf_sp_female

## Fairness Metric: Equalized Odds (using Random Forest)
Let's calculate TPR, TNR, FPR, and FNR for our male and female random forest predictions.

#### True Positives (Equality of Opportunity)

In [ ]:
TPR_rf_male = TPR(test_male_y,rf_male_bin)

In [ ]:
TPR_rf_female = TPR(test_female_y, rf_female_bin)

#### True Negative Rates

In [ ]:
TNR_rf_male = TNR(test_male_y,rf_male_bin)

In [ ]:
TNR_rf_female = TNR(test_female_y,rf_female_bin)

#### False Positive Rates

In [ ]:
FPR_rf_male = FPR(test_male_y, rf_male_bin)

In [ ]:
FPR_rf_female = FPR(test_female_y, rf_female_bin)

#### False Negative Rates

In [ ]:
FNR_rf_male = FNR(test_male_y, rf_male_bin)

In [ ]:
FNR_rf_female = FNR(test_female_y, rf_female_bin)

We'll plot the differences in these values in a little bit. Let's look at our last fairness metric, predictive rate parity, for our Random forest classifier.

## Fairness Metric: Predictive Rate Parity (using Random Forest)


#### Positive Predictive Rate Parity

In [ ]:
# Positive Predictive Rate Parity for male examples
PPR_rf_male = (TPR_rf_male * sum(test_male_y)) / sum(rf_male_bin)

In [ ]:
# Positive Predictive Rate Parity for female examples
PPR_rf_female = (TPR_rf_female * sum(test_female_y)) / sum(rf_female_bin)

#### Negative Predictive Parity

In [ ]:
# Negative Predictive Rate Parity for male examples
NPR_rf_male = (TNR_rf_male * (length(test_male_y) - sum(test_male_y))) / (length(rf_male_bin) - sum(rf_male_bin))

In [ ]:
# Negative Predictive Rate Parity for female examples
NPR_rf_female = (TNR_rf_female * (length(test_female_y) - sum(test_female_y))) / (length(rf_female_bin) - sum(rf_female_bin))

## Now, we'll plot a series of graphs.

In [ ]:
# Let's plot Statistical parity rates for our logistic and random forest classifiers.
mn = [logistic_sp_male, rf_sp_male, logistic_sp_female, rf_sp_female]
sx = repeat(["Men", "Women"], inner = 2)
nam = repeat(["logistic", "random forest"], outer = 2 )

import Pkg; Pkg.add("StatsPlots")
using StatsPlots

groupedbar(nam, mn, group = sx, 
        title = "Statistical Parity", ylim = (0,.2))


In [ ]:
# Let's plot the Statistical parity rate difference (between males and females) for each classifier.

x = ["logistic", "random forest"]
y = [logistic_sp_male-logistic_sp_female, rf_sp_male - rf_sp_female]

b = bar(x,y,alpha=0.4,color="#333333", legend=false, title="Statistical Parity Difference", ylim = (0,.2))

In [ ]:
# Here are the plots for Equalized odds.

mn2 = [TPR_log_male, TPR_rf_male, TPR_log_female, TPR_rf_female]
sx2 = repeat(["Men", "Women"], inner = 2)
nam2 = repeat(["logistic", "random forest"], outer = 2 )

mn3 = [TNR_log_male, TNR_rf_male, TNR_log_female, TNR_rf_female]
sx3 = repeat(["Men", "Women"], inner = 2)
nam3 = repeat(["logistic", "random forest"], outer = 2 )

mn4 = [FPR_log_male, FPR_rf_male, FPR_log_female, FPR_rf_female]
sx4 = repeat(["Men", "Women"], inner = 2)
nam4 = repeat(["logistic", "random forest"], outer = 2 )

mn5 = [FNR_log_male, FNR_rf_male, FNR_log_female, FNR_rf_female]
sx5 = repeat(["Men", "Women"], inner = 2)
nam5 = repeat(["logistic", "random forest"], outer = 2 )

eo1 = groupedbar(nam2, mn2, group = sx2, 
        title = "True Positive Rate")
eo2 = groupedbar(nam3, mn3, group = sx3, 
        title = "True Negative Rate")
eo3 = groupedbar(nam4, mn4, group = sx4, 
        title = "False Positive Rate")
eo4 = groupedbar(nam5, mn5, group = sx5, 
        title = "False Negative Rate")


plot(eo1,eo2,eo3,eo4,layout=(2,2),legend=false, ylim = (0,1))

In [ ]:
# Let's plot the Equalized Odds difference (between males and females) for each classifier.
y1 = [TPR_log_male-TPR_log_female, TPR_rf_male - TPR_rf_female]
y2 = [TNR_log_male-TNR_log_female, TNR_rf_male - TNR_rf_female]
y3 = [FPR_log_male-FPR_log_female, FPR_rf_male - FPR_rf_female]
y4 = [FNR_log_male-FNR_log_female, FNR_rf_male - FNR_rf_female]

b1 = bar(x,y1,alpha=0.4,color="#333333", title="True Positive Rate Difference")
b2 = bar(x,y2,alpha=0.4,color="#333333", title="True Negative Rate Difference")
b3 = bar(x,y3,alpha=0.4,color="#333333", title="False Positive Rate Difference")
b4 = bar(x,y4,alpha=0.4,color="#333333", title="False Negative Rate Difference")

plot(b1,b2,b3,b4,layout=(2,2),legend=false, ylim = (-0.5,0.5))

In [ ]:
# Here are the plots for Predicative Rate Parity.
mn6 = [PPR_male, PPR_rf_male, PPR_female, PPR_rf_female]
sx6 = repeat(["Men", "Women"], inner = 2)
nam6 = repeat(["logistic", "random forest"], outer = 2 )

mn7 = [NPR_male, NPR_rf_male, NPR_female, NPR_rf_female]
sx7 = repeat(["Men", "Women"], inner = 2)
nam7 = repeat(["logistic", "random forest"], outer = 2 )

pr1 = groupedbar(nam6, mn6, group = sx6, 
        title = "Positive Predictive Parity Rate")
pr2 = groupedbar(nam7, mn7, group = sx7, 
        title = "Negative Predictive Parity Rate")


plot(pr1,pr2, layout=(2,1),legend=false, ylim = (0,1))

In [ ]:
# Let's plot the Predicative Rate Parity difference (between males and females) for each classifier.
y5 = [PPR_male-PPR_female, PPR_rf_male - PPR_rf_female]
y6 = [NPR_male-NPR_female, NPR_rf_male - NPR_rf_female]

b5 = bar(x,y5,alpha=0.4,color="#333333", title="Positive Predictive Parity Rate Difference")
b6 = bar(x,y5,alpha=0.4,color="#333333", title="Negative Predictive Parity Rate Difference")


plot(b5,b6,layout=(2,1),legend=false, ylim = (-0.5,0.5))